In [3]:
import pickle
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier

In [11]:
# if os.path.exists("bitcoin_trades.csv"):
#     bitcoinData = pd.read_csv("bitcoin_trades.csv", index_col=0)
#     bitcoinData.index = pd.to_datetime(bitcoinData.index, unit='s')
    
# bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496


In [52]:
import requests

url = "https://data.alpaca.markets/v1beta3/crypto/us/bars?symbols=BTC%2FUSD&timeframe=1Min&start=2023-08-01T00%3A00%3A00Z&limit=10000&sort=asc"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

data = response["bars"]['BTC/USD']

column_mapping = {
    'h': "High",
    "o": "Open",
    "l": "Low",
    "c": "Close",
    "v": "Volume_(BTC)",
    't': 'Timestamp'
}

df = pd.DataFrame(data)
df.rename(columns=column_mapping, inplace=True)

bitcoinData = df

In [53]:
bitcoinData = bitcoinData.dropna()
# bitcoinData.drop(columns=['n','Volume_(BTC)', 'vw'], inplace=True)
bitcoinData.reset_index(drop=True)
bitcoinData.set_index("Timestamp", inplace=True)
bitcoinData.index = pd.to_datetime(bitcoinData.index, unit='s')
bitcoinData

ValueError: non convertible value 2023-08-01T00:00:00Z with the unit 's', at position 0

In [18]:
bitcoinData["next_minute"] = bitcoinData["Close"].shift(-1)

bitcoinData['Up'] = (bitcoinData["Close"] < bitcoinData["next_minute"]).astype(int)
bitcoinData

,Close,High,Low,Open,Timestamp,next_minute,Up
1970-01-01 00:33:40,29408.528688,29415.226000,29398.074050,29415.226000,2023-08-02T09:44:00Z,29419.443800,1
1970-01-01 00:33:41,29419.443800,29419.443800,29405.600397,29409.589400,2023-08-02T09:45:00Z,29423.342100,1
1970-01-01 00:33:42,29423.342100,29428.376569,29419.081640,29421.145049,2023-08-02T09:46:00Z,29414.403905,0
1970-01-01 00:33:43,29414.403905,29424.755735,29414.128924,29422.049269,2023-08-02T09:47:00Z,29425.373350,1
1970-01-01 00:33:44,29425.373350,29432.323450,29416.440041,29418.081400,2023-08-02T09:48:00Z,29429.610500,1
...,...,...,...,...,...,...,...
1970-01-01 02:46:34,29165.602111,29170.623488,29160.115000,29161.795862,2023-08-07T22:49:00Z,29160.392200,0
1970-01-01 02:46:35,29160.392200,29170.867712,29157.394522,29164.651736,2023-08-07T22:50:00Z,29160.917934,1
1970-01-01 02:46:36,29160.917934,29170.602395,29156.804325,29156.804325,2023-08-07T22:51:00Z,29154.145000,0
1970-01-01 02:46:37,29154.145000,29166.113500,29149.290340,29166.113500,2023-08-07T22:52:00Z,29147.530000,0


In [19]:
horizons = [1,3,5,20,60]

predictors = []

for horizon in horizons:
    rolling_averages = bitcoinData.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    bitcoinData[ratio_column] = bitcoinData["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    
    bitcoinData[trend_column] = bitcoinData.shift(1).rolling(horizon).sum()["Up"]
    
    predictors += [ratio_column, trend_column]
    

DataError: Cannot aggregate non-numeric type: object

In [ ]:
bitcoinData = bitcoinData.dropna()
bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,next_minute,Up,Close_Ratio_1,Trend_1,Close_Ratio_3,Trend_3,Close_Ratio_5,Trend_5,Close_Ratio_20,Trend_20,Close_Ratio_60,Trend_60
Timestamp,,,,,,,,,,,,,,,,,,,
2019-01-01 01:20:00,3692.09,3693.35,3692.09,3692.09,0.213983,790.108310,3692.380805,3689.99,0,1.0,1.0,0.999974,1.0,0.999705,2.0,0.999789,11.0,0.999818,27.0
2019-01-01 01:22:00,3690.63,3690.63,3689.99,3689.99,0.011386,42.017298,3690.376431,3688.85,0,1.0,0.0,0.999688,1.0,0.999427,1.0,0.999256,11.0,0.999253,27.0
2019-01-01 01:24:00,3691.18,3691.94,3688.85,3688.85,9.637709,35563.719624,3690.059317,3691.85,1,1.0,0.0,0.999604,1.0,0.999396,1.0,0.999014,10.0,0.998934,27.0
2019-01-01 01:25:00,3691.49,3691.85,3691.49,3691.85,1.113608,4111.264790,3691.841141,3689.74,0,1.0,1.0,1.000439,1.0,1.000278,2.0,0.999861,10.0,0.999749,27.0
2019-01-01 01:26:00,3691.94,3691.94,3685.34,3689.74,2.221163,8187.804861,3686.269281,3691.65,1,1.0,0.0,0.999890,1.0,0.999793,2.0,0.999359,9.0,0.999180,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:55:00,58742.18,58742.18,58714.31,58714.31,2.519999,148004.448110,58731.946927,58686.00,0,1.0,0.0,0.999944,1.0,1.000100,3.0,1.001052,8.0,1.000764,31.0
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,58685.81,0,1.0,0.0,0.999533,1.0,0.999630,2.0,1.000556,8.0,1.000287,30.0
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,58723.84,1,1.0,0.0,0.999837,0.0,0.999675,1.0,1.000503,8.0,1.000291,29.0


In [ ]:
testData = bitcoinData[-100000:]

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Up"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= .65] = 1
    preds[preds < .65] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Up"], preds], axis=1)
    return combined

def backtest(data, model, predictors, start=750000, step=250):
    all_predictions = []
    end = 15

    for i in range(start, data.shape[0], step):
        train = data.iloc[i-2500:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        end -= 1
        print(i)
        if end < 1:
            break
        
    
    return pd.concat(all_predictions)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=250,min_samples_split=75, random_state=1)
train = testData[:-200]
test = testData[-200:]

model.fit(train[predictors], train["Up"])



RandomForestClassifier(min_samples_split=75, n_estimators=250, random_state=1)

In [ ]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["Up"], preds)

0.5581395348837209

In [ ]:
predictions = backtest(bitcoinData, model, predictors)


750000
750250
750500
750750
751000
751250
751500
751750
752000
752250
752500
752750
753000
753250
753500


In [ ]:
predictions["Predictions"].value_counts()

Predictions
0.0    3125
1.0     625
Name: count, dtype: int64

In [ ]:
precision_score(predictions["Up"], predictions["Predictions"])

0.6608

In [ ]:
import pickle

with open("bittybotV1.pkl", 'wb') as f:
    pickle.dump(model,f)

In [ ]:
predictors

['Close_Ratio_1',
 'Trend_1',
 'Close_Ratio_3',
 'Trend_3',
 'Close_Ratio_5',
 'Trend_5',
 'Close_Ratio_20',
 'Trend_20',
 'Close_Ratio_60',
 'Trend_60']